# Sciences po final assingments


install the required packages

In [1]:
import csv
!pip install minet
!pip install ural


  Using cached minet-0.60.2-py3-none-any.whl (180 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached browser_cookie3-0.13.0-py3-none-any.whl
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached json5-0.9.6-py2.py3-none-any.whl (18 kB)
  Using cached cchardet-2.1.7-cp310-cp310-win_amd64.whl
  Using cached quenouille-1.4.2-py3-none-any.whl (16 kB)
  Using cached ebbe-1.8.0-py3-none-any.whl (10 kB)
  Using cached trafilatura-1.2.0-py3-none-any.whl (194 kB)
  Using cached casanova-0.17.1-py3-none-any.whl (20 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached twitwi-0.10.3-py3-none-any.whl (16 kB)
  Using cached keyring-19.2.0-py2.py3-none-any.whl (34 kB)
  Using cached lxml-4.8.0-cp310-cp310-win_amd64.whl (3.6 MB)
  Using cached ndjson-0.3.1-py2.py3-none-any.whl (5.3 kB)
  Using cached ural-0.32.1-py3-none-any.whl (37 kB)
  Using cached dateparser-1.1.1-py2.py3-none-an

You should consider upgrading via the 'C:\Code\SciencesPoFinalAssignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Code\SciencesPoFinalAssignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Scrape the tweets


In [64]:
from minet.twitter import TwitterAPIScraper
import itertools

def chunks(iterable, size=100):
    """Chunks a
     generator into a blocks of size n"""
    iterator = iter(iterable)
    for first in iterator:
        yield itertools.chain([first], itertools.islice(iterator, size - 1))

def get_tweets():
    scraper = TwitterAPIScraper()
    chunk_size=100
    def identity_with_logging(i,x):
        print(i)
        return x
    tweetStream = scraper.search_tweets('(blocage OR occupation) sciencespo')
    return [tweet for i, hundred in enumerate(chunks(tweetStream,chunk_size)) for tweet in identity_with_logging(chunk_size*(i+1),list(hundred))]

tweets = get_tweets()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


Get most retweeted tweet

In [65]:
top_tweet = max(tweets, key=lambda t : t['retweet_count'])
print(f"text: {top_tweet['text']} \nauthor: {top_tweet['user_screen_name']} \nretweets: {top_tweet['retweet_count']}")

"text: 💥📣OCCUPATION EN COURS A SCIENCES PO !  \nAprès l' #ENS et la #Sorbonne, c'est au tour des étudiant-es de SciencesPo à #Paris de se mobiliser. \nNi Le Pen, ni Macron, contre la précarité, le déni écologique et la violation des droits humains, la jeunesse s'organise ! https://twitter.com/sarah_chp/status/1514499001948774403/photo/1 \nauthor: sarah_chp \nretweets: 361"

Get top ten most represented accounts in our corpus

In [ ]:
from functools import reduce # import needed for python3; builtin in python2
from collections import defaultdict

def group(key, seq):
    return reduce(lambda grp, val: grp[key(val)].append(val) or grp, seq, defaultdict(list))

grouped_by_user_name = group(lambda x : x["user_screen_name"], tweets)

In [87]:
top10 = sorted(grouped_by_user_name.items(), key=lambda x : -len(x[1]))[:10]
[(key,len(value)) for key,value in top10]

[('sciencespo', 19),
 ('MatthDes', 18),
 ('DavidLibeau', 17),
 ('LaPeniche', 15),
 ('ubellier', 15),
 ('SieurHibou', 11),
 ('st3vK', 8),
 ('wnewspresse', 7),
 ('ActusNonStop', 6),
 ('unisciencespo', 6)]

Get the top ten users by retweet count

In [93]:
retweet_counts = [(username,sum([tweet['retweet_count'] for tweet in tweets])) for username,tweets in grouped_by_user_name.items()]
top10_total_retweets = sorted(retweet_counts, key=lambda x : -x[1])[:10]
top10_total_retweets


[('sarah_chp', 361),
 ('Anton1Ferreira', 350),
 ('RemyBuisine', 296),
 ('LarrereMathilde', 290),
 ('sciencespo', 272),
 ('LesNews', 239),
 ('LEtudiant_Libre', 223),
 ('Valeurs', 199),
 ('InsoumisJeunes', 181),
 ('UrsusArctos92', 166)]

In [95]:
from statistics import median,mean,stdev
retweet_counts = [x['retweet_count'] for x in tweets]
print(f"min : {min(retweet_counts)}, max : {max(retweet_counts)}, mean : {mean(retweet_counts)}, median : {median(retweet_counts)} stdev : {stdev(retweet_counts)}")


min : 0, max : 361, mean : 5.396591789310612, median : 0 stdev : 23.210612131240275


In [127]:
import csv
rows = [{"url" : key, "count" : len(val)}  for key,val in group(lambda x : x, [link for tweet in tweets for link in tweet['links'] ]).items()]
with open("./zeru_komary_k_snidani.csv", "w", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['url','count'])
    writer.writerows(rows)


get 10 most shared links

In [134]:
most_shared10 = sorted(rows, key=lambda x : -x['count'])
most_shared10[:10]


[{'url': 'http://www.lemonde.fr/enseignement-superieur/article/2013/02/27/sciences-po-occupation-d-un-amphi-contre-la-procedure-de-succession_1840022_1473692.html',
  'count': 19},
 {'url': 'https://youtu.be/hPzR5g-sdoU', 'count': 18},
 {'url': 'https://www.huffingtonpost.fr/2018/04/18/blocage-a-sciencespo-reconnaitrez-vous-la-replique-de-film-culte-citee-par-cet-etudiant_a_23414065/',
  'count': 12},
 {'url': 'http://www.lemonde.fr/campus/article/2018/04/20/a-sciences-po-paris-le-blocage-etudiant-leve-apres-des-negociations_5288336_4401467.html',
  'count': 10},
 {'url': 'https://www.ouest-france.fr/education/universites/blocage-des-universites-sciences-po-bloque-paris-et-rennes-5705238',
  'count': 7},
 {'url': 'http://lemde.fr/13Z0Vjv', 'count': 7},
 {'url': 'https://www.leparisien.fr/elections/presidentielle/presidentielle-occupation-de-la-sorbonne-lens-a-paris-et-de-sciences-po-nancy-par-des-etudiants-en-colere-13-04-2022-MP76WOAVI5ANVISJ7ELSFONBGE.phpnull',
  'count': 6},
 {'url'

In [149]:
list(map(lambda x : x[0], sorted(group(lambda x : x, [hashtag for tweet in tweets for hashtag in tweet['hashtags']]).items(), key=lambda x : -len(x[1]))))[:25]

['sciencespo',
 'blocage',
 'nuitdebout',
 'periscope',
 'paris',
 'occupation',
 'loitravail',
 'agsciencespo',
 'sorbonne',
 'blocus',
 'rennes',
 'occupyscpo',
 'toulouse',
 'tolbiac',
 'occupyboutmy',
 'macron',
 'giletsjaunes',
 'sciencespofacoccupee',
 'agloitravail',
 'sciences',
 'nonalaselection',
 'lgophilippot',
 'étudiants',
 'unef',
 'grenoble']